In [1]:
import sqlite3 as sql
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import numpy as np
import datetime
import plotly.express as px
import joblib
import Funciones as fn
from unidecode import unidecode #  Esta función convierte los caracteres a su forma sin diacríticos, lo que incluye eliminar las tildes.

### **Conectar BD**

In [2]:
###### para ejecutar sql y conectarse a bd #####
conn=sql.connect('BD_insurance2019dataset')
cur=conn.cursor()

In [3]:
### para verificar las tablas que hay disponibles ###
pd.read_sql_query("SELECT * FROM sqlite_master; ",conn)

,type,name,tbl_name,rootpage,sql
0,table,reclamaciones,reclamaciones,2,CREATE TABLE reclamaciones (Reclamacion_Id INT...
1,table,diagnosticos,diagnosticos,3,CREATE TABLE diagnosticos (Diagnostico_Codigo ...
2,table,regional,regional,42,"CREATE TABLE regional (Regional_id INTEGER, Re..."
3,table,genero,genero,43,"CREATE TABLE genero (Sexo_Cd TEXT, Sexo_desc T..."
4,table,sociodemograficas,sociodemograficas,44,CREATE TABLE sociodemograficas (Afiliado_Id IN...
5,table,utilizaciones,utilizaciones,1328,CREATE TABLE utilizaciones (Fecha_Reclamacion ...


**Cargar Tablas**

In [4]:
reclamaciones = pd.read_sql("SELECT * FROM reclamaciones; ",conn)


#### **Tabla Reclamaciones**

In [5]:

reclamaciones.info() # Se cuenta con 2 campos, la reclamacion_id y la reclamacion_desc
reclamaciones.duplicated().sum() #La tabla no tiene duplicados
reclamaciones.head()  # Es una tabla descriptiva de los valores asociados en otras tablas
reclamaciones['Reclamacion_Desc'].value_counts() #Hay valores en minuscula y otros en mayúscula, hay datos con tilde y otros sin tilde
reclamaciones['Reclamacion_Desc'] = reclamaciones['Reclamacion_Desc'].apply(lambda x: unidecode(x) if isinstance(x, str) else x)
reclamaciones['Reclamacion_Desc'] = reclamaciones['Reclamacion_Desc'].str.upper() #Se cambia todos los datos a mayúscula
#reclamaciones['Reclamacion_Desc'].value_counts()
#Este celda de código es para agrupar las categórias que son similares. 
categorizar = {
    'TRATAMIENTO MEDICO AMBULATORIO POR ACCIDENTES' : 'TRATAMIENTO POR ACCIDENTE',
    'TRATAMIENTO MEDICO HOSPITALARIO POR ACCIDENTE' : 'TRATAMIENTO POR ACCIDENTE',
    'TRATAMIENTO QUIRURGICO HOSPITALARIO POR ACCIDENTE' : 'TRATAMIENTO POR ACCIDENTE',
    'CIRUGIA AMBULATORIA POR ACCIDENTE' : 'TRATAMIENTO POR ACCIDENTE',
    'TRATAMIENTO MEDICO HOSPITALARIO POR CANCER Y COMPLICACIONES' : 'TRATAMIENTO POR CANCER',
    'TRATAMIENTO AMBULATORIO POR CANCER': 'TRATAMIENTO POR CANCER',
    'TRATAMIENTO QUIRURGICO HOSPITALARIO POR CANCER Y COMPLICACIONES' : 'TRATAMIENTO POR CANCER',
    'TRATAMIENTO AMBULATORIO POR CANCER': 'TRATAMIENTO POR CANCER',
    'ATENCION DEL PARTO' : 'ATENCION POR EMBARAZO',
    'CONSULTAS PRENATALES' : 'ATENCION POR EMBARAZO',
    'TRATAMIENTO MEDICO HOSPITALARIO POR COMPLICACIONES DEL EMBARAZO' : 'ATENCION POR EMBARAZO',
    'TRATAMIENTO MEDICO HOSPITALARIO POR ENFERMEDAD' : 'TRATAMIENTO POR ENFERMEDAD',
    'TRATAMIENTO MEDICO AMBULATORIO POR ENFERMEDAD EN URGENCIAS' : 'TRATAMIENTO POR ENFERMEDAD',
    'TRATAMIENTO QUIRURGICO HOSPITALARIO POR ENFERMEDAD': 'TRATAMIENTO POR ENFERMEDAD',
    'TRATAMIENTO QUIRURGICO HOSPITALARIO POR ENFERMEDAD CONGENITA' : 'TRATAMIENTO POR ENFERMEDAD',
    'TRATAMIENTOS ESPECIFICOS': 'TRATAMIENTO POR ENFERMEDAD',
    'CIRUGIA AMBULATORIA POR ENFERMEDAD': 'TRATAMIENTO POR ENFERMEDAD',
    'CIRUGIA AMBULATORIA POR ENFERMEDAD CONGENITA': 'TRATAMIENTO POR ENFERMEDAD',
    'DIALISIS': 'TRATAMIENTO POR ENFERMEDAD',
    'EXAMENES DE DIAGNOSTICO' : 'TRATAMIENTO POR ENFERMEDAD',
    'TRATAMIENTO MEDICO HOSPITALARIO SIQUIATRICO' : 'ATENCION-TRATAMIENTO SALUD MENTAL',
    'ATENCION INTEGRAL DE SALUD MENTAL' : 'ATENCION-TRATAMIENTO SALUD MENTAL',
    'CONSULTA EXTERNA' : 'DIAGNOSTICO MEDICO',
    'EXAMENES DE DIAGNOSTICO' : 'DIAGNOSTICO MEDICO',
    'TRATAMIENTOS AMBULATORIOS ESPECIALES' : 'DIAGNOSTICO MEDICO',
    'LABORATORIO CLINICO' : 'DIAGNOSTICO MEDICO',
    'INFILTRACIONES' : 'INYECCIÓN',
    'VACUNACION' : 'INYECCIÓN',
}
reclamaciones['Reclamacion_Desc'] = reclamaciones['Reclamacion_Desc'].replace(categorizar)
reclamaciones['Reclamacion_Desc'].value_counts()
reclamaciones.head(20) #queda pendiente poner el "SIN INFORMACION" con id -1 y agrupar los id que tengan la misma reclamación
#Agrupamos los id que tengan la misma reclamación

# Obtener los valores únicos de la segunda columna (Reclamacion_Desc)
unique_desc = reclamaciones['Reclamacion_Desc'].unique()

# Crear un diccionario para mapear los valores únicos a los números correspondientes
desc_mapping = {desc: i+1 for i, desc in enumerate(unique_desc)}

# Actualizar la segunda columna (Reclamacion_Id) con los valores mapeados
reclamaciones['Reclamacion_Id'] = reclamaciones['Reclamacion_Desc'].replace(desc_mapping)

# Eliminar los valores duplicados en la columna "Reclamacion_Desc"
reclamaciones['Reclamacion_Desc'] = reclamaciones['Reclamacion_Desc'].drop_duplicates()
# Eliminar duplicados en función de la segunda columna
reclamaciones = reclamaciones.drop_duplicates(subset='Reclamacion_Desc')
# Restablecer los índices del DataFrame resultante
reclamaciones = reclamaciones.reset_index(drop=True)

# Eliminar los valores nulos en la columna "Reclamacion_Id"
reclamaciones['Reclamacion_Desc'] = reclamaciones['Reclamacion_Desc'].dropna()
# Eliminar los valores duplicados en la columna "Reclamacion_Id"
reclamaciones['Reclamacion_Id'] = reclamaciones['Reclamacion_Id'].drop_duplicates()
# Eliminar filas con valores nulos
reclamaciones = reclamaciones.dropna()
# Restablecer los índices del DataFrame resultante
reclamaciones = reclamaciones.reset_index(drop=True)


# Mostrar el dataframe resultante
print(reclamaciones)
# Reemplazar el valor de Reclamacion_Id por -1 si Reclamacion_Desc contiene "SIN INFORMACION"
reclamaciones.loc[reclamaciones['Reclamacion_Desc'] == 'SIN INFORMACION', 'Reclamacion_Id'] = -1
reclamaciones.loc[reclamaciones['Reclamacion_Desc'] == 'TRASLADO EN AMBULANCIA', 'Reclamacion_Id'] = 14
reclamaciones.loc[reclamaciones['Reclamacion_Desc'] == 'TRATAMIENTO POR CANCER', 'Reclamacion_Id'] = 15

# Ordenar el DataFrame por la columna 'Reclamacion_Id' de menor a mayor
reclamaciones = reclamaciones.sort_values('Reclamacion_Id')

# Restablecer los índices del DataFrame
reclamaciones = reclamaciones.reset_index(drop=True)

# Eliminar el ".0" de la columna 'Reclamacion_Id'
reclamaciones['Reclamacion_Id'] = reclamaciones['Reclamacion_Id'].astype(int).astype(str)


# Mostrar el dataframe resultante
reclamaciones.head(16)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35 entries, 0 to 34
Data columns (total 2 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Reclamacion_Id    35 non-null     int64 
 1   Reclamacion_Desc  35 non-null     object
dtypes: int64(1), object(1)
memory usage: 688.0+ bytes
    Reclamacion_Id                   Reclamacion_Desc
0              1.0              ATENCION POR EMBARAZO
1              2.0  ATENCION-TRATAMIENTO SALUD MENTAL
2              3.0          TRATAMIENTO POR ACCIDENTE
3              4.0         TRATAMIENTO POR ENFERMEDAD
4              5.0      COBERTURA PROGRAMA PALIATIVOS
5              6.0                       CONCILIACION
6              7.0              CONSULTA DE URGENCIAS
7              8.0                 DIAGNOSTICO MEDICO
8              9.0             FISIOTERAPIA ILIMITADA
9             10.0                         FUNERARIOS
10            11.0                          INYECCIÓN
11         

,Reclamacion_Id,Reclamacion_Desc
0,-1,SIN INFORMACION
1,1,ATENCION POR EMBARAZO
2,2,ATENCION-TRATAMIENTO SALUD MENTAL
3,3,TRATAMIENTO POR ACCIDENTE
4,4,TRATAMIENTO POR ENFERMEDAD
5,5,COBERTURA PROGRAMA PALIATIVOS
6,6,CONCILIACION
7,7,CONSULTA DE URGENCIAS
8,8,DIAGNOSTICO MEDICO
9,9,FISIOTERAPIA ILIMITADA


###Descargas

In [6]:
joblib.dump(reclamaciones,'reclamaciones.pkl') 

['reclamaciones.pkl']